In [3]:
import numpy as np
frame=np.load('frame.npy',allow_pickle=True)

In [4]:
import tensorflow as tf
train_feature=frame[0][0]
for i in range(1,10):
    train_feature = tf.concat([train_feature,frame[i][0]],axis=0)   
for i in range(1,10):
    matrix1 = np.random.random(size=[1,5000000,2])
    train_feature=tf.concat([matrix1,frame[i][0]],axis=0)  

ResourceExhaustedError: OOM when allocating tensor with shape[2,5000000,2] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:ConcatV2] name: concat

In [3]:
train_label=[]
for i in range(10):
    train_label.append(1)
for i in range(10):
    train_label.append(0)

In [4]:
class classification(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1=tf.keras.layers.Conv1D(filters=10,kernel_size=200,padding='same')
        self.pool1=tf.keras.layers.MaxPool1D(pool_size=100)
        self.dense1=tf.keras.layers.Dense(200,activation='tanh')
        self.dense2=tf.keras.layers.Dense(10,activation='relu')
        self.flatten=tf.keras.layers.Reshape(target_shape=(500000,))
        self.output1=tf.keras.layers.Dense(units=1,activation='relu')
    def call(self,inputs):
        temp=self.conv1(inputs)
        temp=self.pool1(temp)
        temp=self.dense1(temp)
        temp=self.dense2(temp)
        temp=self.flatten(temp)
        output=self.output1(temp)
        return output


In [1]:
with tf.device("/gpu:0"):
    learning_rate=0.001
    model=classification()
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    for i in range(3):
        with tf.GradientTape() as tape:
            labels_pred=model(train_feature,training=True)
            loss=tf.reduce_mean(tf.square(train_label - labels_pred))
        grads=tape.gradient(loss,model.trainable_variables)
        optimizer.apply_gradients(grads_and_vars=zip(grads,model.trainable_variables))
        print(loss)


In [ ]:
lab